<a href="https://colab.research.google.com/github/GemmaRagadini/Pokemon_AIF_24_25/blob/main/pokemon-vgc-engine-master/example/Notebook_project_Pokemon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pokemon_Battle PokeBob team

The project focuses on the track proposed in the course, specifically the section related to competitions. Our team selected the task involving the simulation of a Pokémon battle between two teams, each composed of three Pokémon. The battle consists of three matches, with the first player to knock out all three Pokémon of the opposing player declared the winner of the match. The battle is considered concluded when a player wins at least two out of three matches.

The objective of the project was to develop a competitive AI agent in the Pokémon battle environment. We started by challenging a random player who selects its Pokémon moves arbitrarily without any strategic logic. Both players operate under the same conditions, with their teams assigned randomly and regenerated before each challenge, ensuring that any advantages or disadvantages are also determined by chance.
In the following sections, we will explain our approach to solving the task and outline the methodology we adopted, along with the results we obtained.


In [11]:
# Clone the entire repo.
!git clone https://github.com/GemmaRagadini/Pokemon_AIF_24_25.git
%cd Pokemon_AIF_24_25/pokemon-vgc-engine-master
!ls

Cloning into 'Pokemon_AIF_24_25'...
remote: Enumerating objects: 1940, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 1940 (delta 4), reused 0 (delta 0), pack-reused 1909 (from 2)
Receiving objects: 100% (1940/1940), 21.65 MiB | 25.99 MiB/s, done.
Resolving deltas: 100% (481/481), done.
/content/Pokemon_AIF_24_25/pokemon-vgc-engine-master/Pokemon_AIF_24_25/pokemon-vgc-engine-master
CHANGELOG    example	  organization	requirements.txt  test
competition  LICENSE.txt  README.md	setup.py	  vgc


In [12]:
!pip install -r requirements.txt

In [29]:
!pip install virtualenv
!virtualenv amb
!source amb/bin/activate

created virtual environment CPython3.11.11.final.0-64 in 1224ms
  creator CPython3Posix(dest=/content/Pokemon_AIF_24_25/pokemon-vgc-engine-master/Pokemon_AIF_24_25/pokemon-vgc-engine-master/amb, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.3.1, setuptools==75.8.0, wheel==0.45.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [32]:
!source amb/bin/activate

# Related Work

For this project, we decided to explore some existing approaches related to the concepts studied during the course, as well as to develop a custom approach of our own. These approaches were evaluated through a tournament, the details of which will be provided in the corresponding section.

The related work was derived from the course slides and the accompanying textbook, *Artificial Intelligence A Modern Approach - Fourth Edition. Stuart Russer, Peter Norvig*. Specifically, we focused on the section related to game theory (Chapter 6 of the textbook) and examined various approaches to the Minimax algorithm and its variations. This included the implementation of alpha-beta pruning and heuristics such as the killer move heuristic.

# Methodology

To achieve our goal, we decided to implement several algorithms discussed in the related work. We focused our attention on various implementations of the Minimax algorithm using 2 different evaluation functions. In the following sections, we will provide a detailed explanation of each algorithm we implemented.

In [35]:
import math
import hashlib
import numpy as np
import random
from typing import List
from vgc.behaviour import evalFunctions
from vgc.datatypes.Types import PkmStat, PkmType, WeatherCondition
from vgc.datatypes.Objects import Pkm, GameState,PkmType, PkmTeam, PkmStat
from vgc.datatypes.Constants import DEFAULT_PKM_N_MOVES, DEFAULT_PARTY_SIZE, TYPE_CHART_MULTIPLIER, DEFAULT_N_ACTIONS
from vgc.behaviour import BattlePolicy
from copy import deepcopy

# Implemented Policies

We implemented 3 algorithms: classic Minimax, Minimax with alpha-beta pruning and killer heuristic, and My Policy. The third one is the policy that performed the best. The first two were tested with two different evaluation functions, `game_state_eval` and `my_eval_fun`.

The two Minimax algorithms are located in the file `behaviour/otherPolicies.py`, while `MyPolicy` is located in the file `behaviour/myPolicy.py`.


# Minimax

The first implementation is the one obout a classic minimax algorithm with a pre-existent evaluation function called `game_eval`. Here below there is our implementation.


In [36]:
class MyMinimax(BattlePolicy):

    def __init__(self, max_depth: int = 4):
        self.max_depth = max_depth
        self.name = "My Minimax"

    def minimax(self, g, depth, is_maximizing_player):
        """
        Classic minimax
        """

        if depth == 0:
            # Evaluate the current state
            try:
                evaluation = evalFunctions.game_state_eval(g,depth)
            except Exception as e:
                import traceback
                traceback.print_exc()

            return evaluation , None

        if is_maximizing_player: # MAX
            max_eval = float('-inf')
            best_action = None
            try:
                for i in range(DEFAULT_N_ACTIONS):
                    g_copy = deepcopy(g)
                    s, _, _, _, _ = g_copy.step([i, 99])  # The opponent performs an invalid action that does not change the situation
                    if evalFunctions.n_defeated(s[0].teams[0]) > evalFunctions.n_defeated(g.teams[0]):
                        continue # Ignore states where our number of defeated Pokémon increases
                    eval_score, _ = self.minimax(s[0], depth - 1, False)
                    if eval_score > max_eval:
                        max_eval = eval_score
                        best_action = i
            except Exception as e:
                import traceback
                traceback.print_exc()

            return max_eval, best_action

        else:  # MIN
            min_eval = float('inf')
            best_action = None
            try:
                for j in range(DEFAULT_N_ACTIONS):
                    g_copy = deepcopy(g)
                    s, _, _, _, _ = g_copy.step([99, j])
                    if evalFunctions.n_defeated(s[0].teams[1]) > evalFunctions.n_defeated(g.teams[1]):
                        continue
                    eval_score, _ = self.minimax(s[0], depth - 1, True)
                    if eval_score < min_eval:
                        min_eval = eval_score
                        best_action = j
            except Exception as e:
                import traceback
                traceback.print_exc()

            return min_eval, best_action


    def get_action(self, g) -> int:
        """
        Find the best action for MAX
        """
        try:
            _, best_action = self.minimax(g, self.max_depth, True)
        except Exception as e:
            import traceback
            traceback.print_exc()

        return best_action if best_action is not None else 0


In the cell above, we present our first implementation of the Minimax policy, which, in this case, employs `game_state_eval` evaluation functions. This function is described in the *Evaluation Functions* section.

The Minimax implementation is straightforward, comprising a section for the maximizer player and another for the minimizer. The maximizer aims to transition to states where its Pokémon are healthier than the opponent’s Pokémon, while the minimizer seeks to reduce this advantage. Each state is evaluated recursively.

To support these computations, the algorithm uses the `n_defeated` function (in `behaviour/evalFunctions.py`, again in *Evaluation Functions* section), which counts the number of fainted (knocked-out) Pokémon. Additionally, the algorithm determines the next action from the maximizer player’s perspective, as implemented in the `get_action` method.

The default values for the search depth and weights in the evaluation function were determined empirically. Various configurations were tested, and the ones used here were found to deliver the best performance according to our evaluation metrics.


# Minimax with Alpha-Beta Pruning and Killer Move Heuristic

Our second implementation extends the basic Minimax algorithm by incorporating alpha-beta pruning and the killer move heuristic. This implementation was developed to enhance both the performance and efficiency of the Minimax algorithm described above.

The addition of alpha-beta pruning allows the algorithm to eliminate branches in the search tree that cannot influence the final decision, significantly reducing the number of nodes explored. Meanwhile, the killer move heuristic prioritizes moves that are likely to be effective, further optimizing the decision-making process by focusing on promising actions.

The combined use of these techniques aims to not only improve the accuracy of the algorithm but also speed up its execution, enabling faster and more effective decision-making.

In the cells below, we present our implementation of this enhanced Minimax algorithm along with a detailed explanation of how it operates.

In [37]:
class MyMinimaxWithAlphaBetaKiller(BattlePolicy):
    """
    Minimax algorithm with alpha beta pruning and killer moves optimization
    """
    def __init__(self, max_depth: int = 5):
        self.max_depth = max_depth
        self.name = "Minimax with pruning alpha beta killer"
        self.killer_moves = {depth: [] for depth in range(max_depth + 1)}

    def minimax(self, g, depth, alpha, beta, is_maximizing_player):
        if depth == 0:
            try:
                evaluation = evalFunctions.game_state_eval(g,depth)
            except Exception as e:
                import traceback
                traceback.print_exc()
            return evaluation, None

        if is_maximizing_player:
            max_eval = float('-inf')
            best_action = None

            # Possible actions
            moves = list(range(DEFAULT_N_ACTIONS))
            # prioritizing killer moves
            killer_moves = self.killer_moves.get(depth, [])
            moves = sorted(moves, key=lambda move: move in killer_moves, reverse=True)

            try:
                for i in moves:
                    g_copy = deepcopy(g)
                    s, _, _, _, _ = g_copy.step([i, 99])
                    if evalFunctions.n_defeated(s[0].teams[0]) > evalFunctions.n_defeated(g.teams[0]):
                        continue

                    eval_score, _ = self.minimax(s[0], depth - 1, alpha, beta, False)
                    if eval_score > max_eval:
                        max_eval = eval_score
                        best_action = i

                    alpha = max(alpha, eval_score)
                    if beta <= alpha:
                        # update killer moves
                        if i not in self.killer_moves[depth]:
                            self.killer_moves[depth].append(i)
                            if len(self.killer_moves[depth]) > 2:
                                self.killer_moves[depth].pop(0)
                        break
            except Exception as e:
                import traceback
                traceback.print_exc()

            return max_eval, best_action

        else:
            min_eval = float('inf')
            best_action = None

            moves = list(range(DEFAULT_N_ACTIONS))

            try:
                killer_moves = self.killer_moves.get(depth, [])
                moves = sorted(moves, key=lambda move: move in killer_moves, reverse=True)
            except Exception as e:
                import traceback
                traceback.print_exc()

            try:
                for j in moves:
                    g_copy = deepcopy(g)
                    s, _, _, _, _ = g_copy.step([99, j])
                    if evalFunctions.n_defeated(s[0].teams[1]) > evalFunctions.n_defeated(g.teams[1]):
                        continue

                    eval_score, _ = self.minimax(s[0], depth - 1, alpha, beta, True)
                    if eval_score < min_eval:
                        min_eval = eval_score
                        best_action = j

                    beta = min(beta, eval_score)
                    if beta <= alpha:
                        if j not in self.killer_moves[depth]:
                            self.killer_moves[depth].append(j)
                            if len(self.killer_moves[depth]) > 2:
                                self.killer_moves[depth].pop(0)
                        break
            except Exception as e:
                import traceback
                traceback.print_exc()

            return min_eval, best_action

    def get_action(self, g) -> int:
        try:
            _, best_action = self.minimax(g, self.max_depth, float('-inf'), float('inf'), True)
        except Exception as e:
                import traceback
                traceback.print_exc()

        return best_action if best_action is not None else 0


Here we illustrate the key components of this algorithm.

Alpha-Beta Pruning is a fundamental optimization technique used in Minimax algorithms as we said before. It reduces the number of nodes evaluated in the game tree by eliminating branches that cannot influence the final decision. Specifically:
Alpha represents the best score achievable by the maximizing player, ensuring the current node's value is not less than this threshold.
Beta represents the best score achievable by the minimizing player, ensuring the current node's value does not exceed this threshold. By comparing node evaluations against these bounds, the algorithm can skip unnecessary evaluations and focus on the most promising branches.
The Killer Move Heuristics prioritize actions (or moves) that previously led to a cutoff during Alpha-Beta Pruning at the same depth. These "killer moves" are stored in a depth-specific list and revisited with high priority in subsequent iterations. The rationale is that actions causing significant pruning in similar scenarios are likely to be effective again, reducing the time spent on less impactful moves.

The algorithm explores the game tree up to a predefined depth, *max_depth*. At this point, it evaluates the game state using a heuristic evaluation function,the same of the latest implementation of Minimax. This depth limit balances the trade-off between computational feasibility and decision quality and it was selected in a empirical way.

**Algorithm Workflow**

The algorithm begins by initializing necessary parameters, including the maximum search depth and a set of *killer moves* for each depth level. These killer moves are essentially a list of actions that have proven effective in causing cutoffs during previous searches. By storing these moves, the algorithm can prioritize their exploration in subsequent iterations, aiming to reduce unnecessary computations.

Once initialized, the algorithm proceeds to explore the game tree using the Minimax method. This exploration is depth-limited, meaning the algorithm only evaluates the game tree to a specific depth (*max_depth*) to keep the computation manageable. At the root node, the algorithm decides whether it is currently the turn of the maximizing or minimizing player and selects actions accordingly.

For the maximizing player, the goal is to find the action that yields the highest evaluation score and where the number of fainted pokemon is higher for the opponent, representing the most advantageous outcome. Conversely, for the minimizing player, the focus is on selecting the action that minimizes the evaluation score, simulating an opponent trying to counteract the maximizing player’s strategies. At each level of the game tree, Alpha and Beta values are used to dynamically track the best and worst outcomes possible for the respective players. These values guide the pruning process, helping the algorithm decide when to stop exploring certain branches.

The algorithm introduces a prioritization mechanism at this stage by sorting available actions based on the killer moves. If a move caused a cutoff at the same depth in a previous search, it is considered likely to do so again and is explored first. This ensures the algorithm focuses on the most promising actions early on, increasing the chances of quickly achieving cutoffs. For example, if the algorithm identifies a move that significantly improves the maximizing player’s position, it will immediately consider pruning all remaining moves that cannot surpass this outcome.

As the game tree is traversed, the algorithm continuously updates the Alpha and Beta values. When a node’s evaluation score falls outside the range defined by Alpha and Beta, the algorithm terminates further exploration of that branch. This process, known as pruning, saves computational resources by avoiding the evaluation of irrelevant or unpromising paths. If a cutoff occurs, the current move is added to the list of killer moves for the corresponding depth, ensuring that it will be prioritized in future iterations.

Once all possible actions are evaluated, the algorithm determines the best action for the maximizing player at the root node.

# Evaluation Functions

The evaluation function plays a central role in determining the quality of a game state during a Pokémon battle simulation.

The purpose of the evaluation function is to return a numerical score that represents the desirability of the current game state. Higher scores indicate more favorable conditions for the agent, while lower scores highlight disadvantages.

`game_state_eval` is the pre-existent evaluation function, it encourages states where the player’s active Pokémon (mine) has higher HP relative to its maximum HP and penalizes states where the opponent’s active Pokémon (opp) has high HP.
Finally it adds a penalty proportional to the search depth to prioritize faster victories.
Our results demonstrate that it provides a balanced implementation. However, it is not the most effective approach we encountered.

To try to improve the performance of the algorithms, we implemented another evaluation function, `my_eval_fun` that builds upon the previous one.
The function achieves this by evaluating three core aspects:

*Type Compatibility*: The function evaluates the matchup between the agent's active Pokémon and the opponent's active Pokémon. This includes assessing whether the agent's Pokémon has moves that are particularly effective against the opponent. For example, if the agent's Pokémon has a "super effective" move available, the function assigns a higher score to reflect this strategic advantage.

*Health Points (HP) Analysis*: we used the base evaluation function, `game_state_eval`, to measure the state of the Pokémon's HP as we did in the previous minimax implementation. This component assesses the agent’s ability to maintain survivability by factoring in both the agent's and the opponent's remaining HP.

*Damage Potential*: The function estimates the maximum damage the agent's Pokémon can inflict on the opponent in the current turn. This is done by considering the Pokémon's strongest available move and adjusting for various factors such as type effectiveness, stat boosts or reductions, and environmental conditions like weather. The damage is then normalized to ensure it aligns proportionally with the other components of the evaluation.

These values are combined into a single score using weighted contributions:
the damage potential and matchup analysis are scaled to ensure they have comparable influence on the final score.
Health points are adjusted with an offset to reflect their importance without overpowering the other factors.
Together, these components create a balanced evaluation of the game state.


In [38]:
def game_state_eval(s: GameState, depth):
    """
    Pre-existing evaluation function
    """
    mine = s.teams[0].active
    opp = s.teams[1].active
    return mine.hp / mine.max_hp - 3 * opp.hp / opp.max_hp - 0.3 * depth

def estimate_damage(move_type: PkmType, pkm_type: PkmType, move_power: float, opp_pkm_type: PkmType,
                    attack_stage: int, defense_stage: int, weather: WeatherCondition) -> float:
    """
    (pre-existing)
    Estimate the damage that the move moveType deals to the opposing Pokémon, taking into account:
    - types of both Pokémon
    - type of the move
    - attack and defense stats
    - weather
    - move power
    """
    # Bonus for same-type move used by the Pokémon
    stab = 1.5 if move_type == pkm_type else 1.
    # Favorable weather
    if (move_type == PkmType.WATER and weather == WeatherCondition.RAIN) or (
            move_type == PkmType.FIRE and weather == WeatherCondition.SUNNY):
        weather = 1.5
    # Unfavorable weather
    elif (move_type == PkmType.WATER and weather == WeatherCondition.SUNNY) or (
            move_type == PkmType.FIRE and weather == WeatherCondition.RAIN):
        weather = .5
    else:
        weather = 1.
    # relative level attack - defense
    stage_level = attack_stage - defense_stage # in [-10,10]
    # Multiplier that increases linearly for positive values and applies fractional reductions for negative values
    stage = (stage_level + 2.) / 2 if stage_level >= 0. else 2. / (np.abs(stage_level) + 2.) # in [0,6]
    # damage estimation
    damage = TYPE_CHART_MULTIPLIER[move_type][opp_pkm_type] * stab * weather * stage * move_power # Approximately 140
    return damage

def my_eval_fun(s:GameState, depth):
    """
    Our evaluation function that considers the compatibility between Pokémon,
    the game_state_eval with respect to hp, and the ability to deal damage
    """
    my_active = s.teams[0].active
    opp_active = s.teams[1].active
    attack_stage = s.teams[0].stage[PkmStat.ATTACK]
    defense_stage = s.teams[1].stage[PkmStat.DEFENSE]
    matchup = examine_matchup(my_active.type, opp_active.type, list(map(lambda m: m.type, my_active.moves))) # in [0,2]
    eval_hp = game_state_eval(s,depth) + 4 # in [0-5]
    max_damage = maxDamage(my_active, opp_active.type, attack_stage, defense_stage, s.weather) # in [0,140]
    return max_damage/70 + matchup/2 + eval_hp


def maxDamage(my_active: Pkm, opp_active_type:PkmType, attack_stage: int, defense_stage: int,weather: WeatherCondition ):
    """
    Returns the maximum damage the active Pokémon can deal to the opponent with a move
    """
    mvs_damage = []
    # estimate the damage for each move of my Pokémon
    for m in my_active.moves:
        mvs_damage.append(estimate_damage(m.type,my_active.type, m.power, opp_active_type ,attack_stage, defense_stage, weather))
    return np.max(mvs_damage)


def examine_matchup(pkm_type: PkmType, opp_pkm_type: PkmType, moves_type: List[PkmType]) -> float:
    """
    Evaluates the matchup between the active Pokémon and the opponent's Pokémon,
    considering the types of the Pokémon and the available moves.
    """
    for mtype in moves_type: # search for super effective move
        if TYPE_CHART_MULTIPLIER[mtype][pkm_type] == 2.0:
            return 2.0  # if there is a super effective move
    # Consider only the evaluation based on the Pokémon's type
    return TYPE_CHART_MULTIPLIER[opp_pkm_type][pkm_type]

def n_defeated(t: PkmTeam):
    """
    compute the number of defeated pokemon in the team
    """
    fainted = 0
    fainted += t.active.hp == 0
    if len(t.party) > 0:
        fainted += t.party[0].hp == 0
    if len(t.party) > 1:
        fainted += t.party[1].hp == 0
    return fainted

# Custom Policy

The third and last approch was the one about a our custom Policy, implemented in `behaviour/myPolicy.py`.

In [39]:
class MyPolicy(BattlePolicy):

    def __init__(self):
        self.hail_used = False
        self.sandstorm_used = False
        self.name = "My Policy"

    def assess_damages(self, active_pkm: Pkm, opp_pkm_type: PkmType, attack_stage: int, defense_stage: int, weather: WeatherCondition)-> int:
        # moves evaluation
        damages: List[float] = []
        for move in active_pkm.moves:
            damages.append(evalFunctions.estimate_damage(move.type, active_pkm.type, move.power, opp_pkm_type, attack_stage,
                                          defense_stage, weather))
        return damages


    def get_action(self, g: GameState) -> int:
        # my team
        my_team = g.teams[0]
        active_pkm = my_team.active
        bench = my_team.party
        my_attack_stage = my_team.stage[PkmStat.ATTACK]

        # opposite team
        opp_team = g.teams[1]
        opp_active_pkm = opp_team.active
        opp_defense_stage = opp_team.stage[PkmStat.DEFENSE]

        # weather
        weather = g.weather.condition

        try:
            # estimate of the damage of each move
            damages = self.assess_damages(active_pkm, opp_active_pkm.type, my_attack_stage, opp_defense_stage, weather)
            move_id = int(np.argmax(damages))
        except Exception as e:
            import traceback
            traceback.print_exc()

        # if it eliminates the opponent or the move type is super effective, use it immediately
        if (damages[move_id] >= opp_active_pkm.hp) or (damages[move_id] > 0 and TYPE_CHART_MULTIPLIER[active_pkm.moves[move_id].type][opp_active_pkm.type] == 2.0) :
            return move_id
        try:
            defense_type_multiplier = evalFunctions.examine_matchup(active_pkm.type, opp_active_pkm.type,
                                                    list(map(lambda m: m.type, opp_active_pkm.moves)))
        except Exception as e:
            import traceback
            traceback.print_exc()

        if defense_type_multiplier <= 1.0:
            return move_id

        # Consider the Pokémon switch
        matchup: List[float] = []
        not_fainted = False

        try:
            for j in range(len(bench)):
                if bench[j].hp == 0.0:
                    matchup.append(0.0)
                else:
                    not_fainted = True
                    matchup.append(
                        evalFunctions.examine_matchup(bench[j].type, opp_active_pkm.type, list(map(lambda m: m.type, bench[j].moves))))

            best_switch_matchup = int(np.max(matchup))
            best_switch = np.argmax(matchup)
            current_matchup = evalFunctions.examine_matchup(active_pkm.type, opp_active_pkm.type,list(map(lambda m: m.type, active_pkm.moves)))
        except Exception as e:
            import traceback
            traceback.print_exc()

        if not_fainted and best_switch_matchup >= current_matchup+1:
            return best_switch + 4

        return move_id

The policy is designed to make decisions about the actions the player's team should take based on an evaluation of damage, the Pokémon’s types, and various other battle conditions, such as weather. Here's an overview of its functionality:

*Initialization*: The class initializes two flags, hail_used and sandstorm_used, to track if certain weather conditions have already been activated during the battle.

*Damage Assessment* (`assess_damages`):
This method evaluates the potential damage of each move available to the active Pokémon, using `evalFunctions.estimate_damage`. It calculates the damage based on various factors such as:

  - The Pokémon's attack and the opponent's defense stages.
  - The Pokémon types and move types.
  - Weather conditions that might affect damage output.

It then returns a list of damage estimates for all available moves.

*Action Selection* (`get_action`):
This is the primary method used to decide the next action. It follows a series of steps to make the decision:

  - Team Setup: It first extracts the active Pokémon from the player's team and the opponent's team.
  - Weather Condition: It retrieves the current weather condition, which can influence the effectiveness of moves.
  - Damage Calculation: Using the `assess_damages` method, it calculates the potential damage for each move and selects the move with the highest estimated damage.

*Move Selection Logic*: If a move will eliminate the opponent or if it is "super effective" (based on the type chart), it is selected immediately. If the active Pokémon is at a disadvantage in terms of move effectiveness, the policy evaluates to switch to a Pokémon from the bench (the reserve Pokémon) that has a better matchup against the opponent's active Pokémon. This is determined by evaluating the compatibility between each Pokémon on the bench and the opponent’s active Pokémon. The policy uses a comparison between the matchups of the active Pokémon and each bench Pokémon, selecting the Pokémon that has a significantly better type advantage.

*Pokémon Switch Consideration*: If there is at least one Pokémon on the bench that has a favorable matchup compared to the active Pokémon, the policy will switch to that Pokémon. This is done by comparing the matchup values, and if the bench Pokémon’s matchup score is sufficiently better (greater than the current active Pokémon's matchup by at least 1), it will choose to switch to that Pokémon. When switching Pokémon, "time" is lost in battle, so the policy chooses to do so only if the gain is considerable.

The policy uses the `evaluate_matchup` function to assess the compatibility between Pokémon types based on their moves, making it a dynamic policy that adapts to the strengths and weaknesses of the opposing team.

This battle policy uses a combination of damage estimation, type advantages, and Pokémon switching strategies to make optimal decisions, ensuring that the player can both maximize damage and strategically manage their team for better performance in battle.

# Performance Evaluation

We implemented several approaches and first thing we decided to have each agent battle against the Random agent. This allowed us to demonstrate that each policy outperforms the Random agent.

We chose to use a random team for each player, generated for every match, so that with a large number of executions, the influence of the chosen team on the evaluation of the algorithm's performance decreases.

For the second evaluation, we organized a tournament involving these players:

- *My Policy Player*
- *Random Player*
- *Minimax Player*
- *Minimax with Alpha-Beta Pruning and Killer Move Heuristic Player*
- *Minimax Player using `my_eval_fun`*
- *Minimax with Alpha-Beta Pruning and Killer Move Heuristic Player using `my_eval_fun`*



In [40]:
import time as t
from example.Example_Competitor import MyCompetitor0, MyCompetitor1, MyCompetitor2, MyCompetitor3, MyCompetitor4, MyCompetitor5
from vgc.balance.meta import StandardMetaData
from vgc.competition.Competitor import CompetitorManager
from vgc.ecosystem.BattleEcosystem import BattleEcosystem
from vgc.util.generator.PkmRosterGenerators import RandomPkmRosterGenerator
from vgc.util.generator.PkmTeamGenerators import RandomTeamFromRoster
import sys
import random

N_PLAYERS = 2


def main():

    roster = RandomPkmRosterGenerator().gen_roster()
    meta_data = StandardMetaData()
    le = BattleEcosystem(meta_data, debug=True)
    n_epochs = 10

    times1, wins_0_different,rate1, policy_name1 = SingleCombat(n_epochs,le,roster)

    print(f"Player 0 con {policy_name1} ha vinto: {wins_0_different} partite su {n_epochs}, win rate {rate1}, tempo impiegato {times1}")


def SingleCombat(n_epochs, le:BattleEcosystem,roster):

    wins_player0 = 0
    wins_player1 = 0
    start_time = t.time()

    for i in range(n_epochs):
        print(f"Epoch {i} of {n_epochs}")
        cm1 = CompetitorManager(MyCompetitor0("Player 0"))
        cm1.team = RandomTeamFromRoster(roster).get_team()
        le.register(cm1)
        cm2 = CompetitorManager(MyCompetitor1("Player 1"))
        cm2.team = RandomTeamFromRoster(roster).get_team()
        le.register(cm2)
        # single epoch
        le.run(1)
        # counts wins of Player 0 and Player 1
        wins_player0 += le.win_counts[cm1]
        wins_player1 += le.win_counts[cm2]
        le.unregister(cm1)
        le.unregister(cm2)
    end_time = t.time()
    time = end_time-start_time

    return time,wins_player0,(wins_player0/n_epochs)*100, cm1.competitor.battle_policy.name



def Tournament():

    print("Let the Tournament begin")

    roster = RandomPkmRosterGenerator().gen_roster()
    meta_data = StandardMetaData()

    # trainers
    Pokebob = CompetitorManager(MyCompetitor0("Player 0"))
    randomtrainer = CompetitorManager(MyCompetitor1("Player 1"))
    minimax = CompetitorManager(MyCompetitor2("Player 2"))
    minimax_killer= CompetitorManager(MyCompetitor3("Player 3"))
    minimax_my_eval = CompetitorManager(MyCompetitor4("Player 4"))
    minimax_killer_my_eval = CompetitorManager(MyCompetitor5("Player 5"))
    trainers= [Pokebob, randomtrainer, minimax,minimax_killer, minimax_my_eval, minimax_killer_my_eval]

    scores = [0] * len(trainers)

    n_epochs = 10 # number epochs for each match

    # italian tournament with each trainer having a randomly generated team for each battle
    print("Tournament")
    for i in range(len(trainers)):
        for j in range(i + 1, len(trainers)):  # Avoid duplicate matches
            #print(f"Begin Match between {trainers[i].competitor.battle_policy.name} and {trainers[j].competitor.battle_policy.name}")
            wins_player0 = 0
            wins_player1 = 0
            for k in range(n_epochs):
                #print(f"Epoch {k} of {n_epochs}")
                try:
                    le = BattleEcosystem(meta_data, debug=True)
                    trainers[i].team = RandomTeamFromRoster(roster).get_team()
                    trainers[j].team = RandomTeamFromRoster(roster).get_team()
                    le.register(trainers[i])
                    le.register(trainers[j])
                    # match
                    le.run(1)
                    wins_player0 += le.win_counts[trainers[i]]
                    wins_player1 += le.win_counts[trainers[j]]
                    le.unregister(trainers[i])
                    le.unregister(trainers[j])
                except Exception as e:
                    import traceback
                    traceback.print_exc()

            scores[i] += wins_player0
            scores[j] += wins_player1
            #print(f"Match Results: {trainers[i].competitor.battle_policy.name} won {wins_player0}, {trainers[j].competitor.battle_policy.name} won {wins_player1}")

    # sorted ranking
    ranking = sorted(zip(trainers, scores), key=lambda x: x[1], reverse=True)
    print("\nRanking tournament:\n")
    for i, (trainer, score) in enumerate(ranking, start=1):
        print(f"{i}. {trainer.competitor.battle_policy.name} - {score} punti")

# Tournament

We organized a round-robin tournament, in which each player fights against every other player and earns one point for each match won. Each confrontation consists of N matches, each of which is made up of 3 individual battles. The number of matches won is counted, and the player rankings are created. We show the results for the tournament with N = 10.

In [ ]:
Tournament()

Let the Tournament begin
Tournament

Ranking tournament:

1. My Policy - 36 punti
2. My Minimax - 32 punti
3. Minimax with pruning alpha beta killer - 30 punti
4. Minimax with pruning alpha beta killer and my eval - 25 punti
5. My Minimax with my eval - 22 punti
6. Random Player - 5 punti


As we can from the result our policy wins the tournament with the most winned matches. Also Minimax and Minimax with alpha beta pruning and killer move heuristic perform very well and they beat the minimax with our evaluation function whuch consider also the power of a move of a pokemon. In this tournament the fact that the implementation of minimax with our evaluation function dosen't perform so well can be by the fact that the team assigned to the two player (Minimax and Minimax with my eval) were to favorable to the Minimax player (even if they are selected in a random way)

# Conclusion

The result from the the simulations of the battle was allineated with what we thought. In fact every player with a policy different from the random one was able to defeat the random player, not always with outstanding results but they culd beat the random player. We implemented an algorithm that wins a very high percentage of matches (almost 100%) against the random agent and, as seen from the tournament, achieves very satisfactory results against the other implemented algorithms.

The difference in performance with the change of evaluation function in the other two algorithms does not seem to be very impactful, as the two functions do not have fundamental differences in their approach. The major difference is with MyPolicy, as can be seen from the tournament results.  

We believe that MyPolicy performs better than the various Minimax algorithms in this context because, with few game variables and possible strategies, this allowed us to be very specific in the implementation of the algorithm, which adapts well to the very specific situations of the game.

MyPolicy performs well even against some of the agents from the VGC competitions of 2023 and 2024, we have shown these results in Appendix.


# Appendix

We tested the performance of MyPolicy with challenges consisting of 50 epochs against 3 agents from the 2023 and 3 from 2024 competitions. Here are the results obtained:

- MyPolicy vs MySubmissionMR-M.Ruppert: 38-12
- MyPolicy vs vgc_weiyi_yen-Wei Yi Yen: 49-1
- MyPolicy vs WiktorBukowski-Wiktor Bukowski: 35-15
- MyPolicy vs campiao-Pedro Campião: 30-20
- MyPolicy vs Bot4TeamBuildPolicy-Anja Ka: 23-27
- MyPolicy vs MyPokemon-hgvbhjvcfg gh: 34-16